In [ ]:
import torch.nn as nn

class ConvBlock(nn.Module):
    def __init__(self, C_in, C_out, kernel_size, stride):
        super(ConvBlock, self).__init__()
        self.block = nn.Sequential(
                          nn.Conv2d(in_channels=C_in, out_channels=C_out, kernel_size=kernel_size, stride=stride),
                          nn.ReLU(),
                          nn.MaxPool2d(2))
        
    def forward(self, x):
        return self.block(x)
    
class BaselineModel(nn.Module):
    def __init__(self, num_blocks):
        super(BaselineModel, self).__init__()
        layers = []
        channels = [3, 64, 128, 256]
        for i in range(num_blocks):
            layers.append(ConvBlock(C_in=channels[i], C_out=in_channels[i+1], kernel_size=5, stride=1))
        layers.append(nn.Linear(channels[-1],512))
        self.net = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.net(x)